# Тема "Регрессионные методы"

## Подготовка данных

In [1]:
import pandas as pd #импортируем уже знакомую нам библиотеку Pandas

In [2]:
df = pd.read_csv('Модуль 2_Тема 1. Titanic.csv') #загружаем данные из файла, это снова данные о пассажирах Титаника

In [3]:
df.head() #смотрим на загруженные данные

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df = df.drop(['Cabin', 'Name', 'Sex', 'Ticket', 'Embarked'], axis=1) 
# Для использования методов линейной или логистической регрессии можно использовать только числовые данные, поэтому
# самое простое что мы можем сделать это удалить столбцы с тектовыми данными (однако таr мы теряем много информации об объектах).
# Те столбцы, которые нужно удалить видно в info.
# Столбцы с текстовыми данными обозначены как 'object'

In [6]:
df = df[~df['Age'].isnull()] # при обучении машины нельзя использовать данные с пропусками. Из info видно, что возраст известен только
                            # для 714 человек, поэтому удаляем все строки с пропусками.

In [7]:
df.shape

(714, 7)

In [8]:
df.info() #теперь видно, что все в порядке и предварительная обработка данных закончена

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  714 non-null    int64  
 1   Survived     714 non-null    int64  
 2   Pclass       714 non-null    int64  
 3   Age          714 non-null    float64
 4   SibSp        714 non-null    int64  
 5   Parch        714 non-null    int64  
 6   Fare         714 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 44.6 KB


In [9]:
y = df['Survived'] # это и есть наш target или целевая функция или выход.

## Переходим к машинному обучению

Логистическая регрессия – это разновидность множественной регрессии, общее назначение которой состоит в анализе связи между несколькими независимыми переменными (называемыми также регрессорами или предикторами) и зависимой переменной. Бинарная логистическая регрессия, как следует из названия, применяется в случае, когда зависимая переменная является бинарной (т.е. может принимать только два значения). Иными словами, с помощью логистической регрессии можно оценивать вероятность того, что событие наступит для конкретного испытуемого (больной/здоровый, возврат кредита/дефолт и т.д.).

Например, если рассматривается исход по займу, задается переменная y со значениями 1 и 0, где 1 означает, что соответствующий заемщик расплатился по кредиту, а 0, что имел место дефолт. Однако здесь возникает проблема: множественная регрессия не "знает", что переменная отклика бинарна по своей природе. Это неизбежно приведет к модели с предсказываемыми значениями большими 1 и меньшими 0. Но такие значения вообще не допустимы для первоначальной задачи. Таким образом, множественная регрессия просто игнорирует ограничения на диапазон значений для y.

Для решения проблемы задача регрессии может быть сформулирована иначе: вместо предсказания бинарной переменной, мы предсказываем непрерывную переменную со значениями на отрезке [0,1] при любых значениях независимых переменных. Это достигается применением следующего регрессионного уравнения (логит-преобразование):

![Image](https://wikimedia.org/api/rest_v1/media/math/render/svg/bf2ce67feccc5082cf74bf86a55abf0ddd272e9c)
где p – вероятность того, что произойдет интересующее событие; e – основание натуральных логарифмов 2,71…; y – стандартное уравнение регрессии.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Sigmoid-function-2.svg/2000px-Sigmoid-function-2.svg.png" alt="Image">

In [10]:
from sklearn.linear_model import LogisticRegression  # импортируем модель машинного обучения
model = LogisticRegression(max_iter=10000)            # инициализируем модель
model.fit(df, y)                                     # обучаем модель
predict = model.predict(df)                          # делаем предсказание

In [11]:
predict #выводим полученный результат.
#каждому из 714 объектов, это пассажиры Титаника ставится в соответствие выжил он или нет. Модель принимает решения
#на основе признаков (атрибутов) которые мы ей дали в процессе обучения model.fit(df, y)

array([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,

Теперь нужно узнать насколько хорошо наша модель предсказывает, ведь мы же знаем ответы, они находятся в столбце df['Survived']

In [12]:
list(zip(y, predict)) # сравним глазами, правда это очень неудобно.

[(0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 

Посчитаем сколько ответов отличается

In [13]:
sum(abs(y - predict))  # как оказалось, наша модель предсказывает все абсолютно верно, но это только потому, 
#что мы совершили ошибку

0

In [14]:
df.head().T #еще раз смотрим на данные на которых обучается модель.
# транспонируем нашу таблицу, чтобы было удобнее смотреть

,0,1,2,3,4
PassengerId,1.00,2.0000,3.000,4.0,5.00
Survived,0.00,1.0000,1.000,1.0,0.00
Pclass,3.00,1.0000,3.000,1.0,3.00
Age,22.00,38.0000,26.000,35.0,35.00
SibSp,1.00,1.0000,0.000,1.0,0.00
Parch,0.00,0.0000,0.000,0.0,0.00
Fare,7.25,71.2833,7.925,53.1,8.05


In [15]:
# Мы сами же передавли машине ответы (колонка Survived) и на основе этих ответов машина обучалась.
# Поэтому она и смогла предсказать все абсолютно верно.
# Для того чтобы модель была корректной и действительно происходило машинное обучение,
# столбец с ответами нужно удалить.
# Только в этом случае моедль сможет адекватно предсказывать результаты для новых объектов.
df = df.drop('Survived', axis=1) #удаляем ответы из датафрейма

In [16]:
# Повторяем код обучения и предсказания
model = LogisticRegression()                     # инициализируем модель
model.fit(df, y)                                 # обучаем модель
predict = model.predict(df)                      # делаем предсказание

In [17]:
sum(abs(y - predict))  #теперь модель кажется более адекватной

208

In [18]:
#Каким образом можно оценить качество модели?
#Это делается при помощи метрик.

Как правило, результаты решения двоичной классификации помечены как позитивные (положительные) и негативные (отрицательные). 

ROC-кривая (Receiver Operator Characteristic) – кривая, которая наиболее часто используется для представления результатов бинарной классификации в машинном обучении. Название пришло из систем обработки сигналов. Поскольку классов два, один из них называется классом с положительными исходами, второй – с отрицательными исходами. ROC-кривая показывает зависимость количества верно классифицированных положительных примеров от количества неверно классифицированных отрицательных примеров. В терминологии ROC-анализа первые называются истинно положительным, вторые – ложно отрицательным множеством. При этом предполагается, что у классификатора имеется некоторый параметр, варьируя который, мы будем получать то или иное разбиение на два класса. Этот параметр часто называют порогом, или точкой отсечения (cut-off value). В зависимости от него будут получаться различные величины ошибок I и II рода.

В логистической регрессии порог отсечения изменяется от 0 до 1 – это и есть расчетное значение уравнения регрессии. Будем называть его рейтингом.

Для понимания сути ошибок I и II рода рассмотрим таблицу из четырех ячеек, называемую таблицей сопряженности или матрицей ошибок (confusion matrix), которая строится на основе результатов классификации модели и фактической (объективной) принадлежностью примеров к классам.

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/440px-Precisionrecall.svg.png)

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/Sensitivity_and_specificity.svg/525px-Sensitivity_and_specificity.svg.png)

TP (True Positives) – верно классифицированные положительные примеры (так называемые истинно положительные случаи) (то есть объекты, которые были классифицированы как положительные и действительно являются положительными (принадлежащими к данному классу).

TN (True Negatives) – верно классифицированные отрицательные примеры (истинно отрицательные случаи) (то есть объекты, которые были классифицированы как отрицательные и действительно отрицательные (не принадлежат к данному классу).

FN (False Negatives) – положительные примеры, классифицированные как отрицательные (ошибка I рода). Это так называемый "ложный пропуск" – когда интересующее нас событие ошибочно не обнаруживается (ложно отрицательные примеры) (то есть объекты, которые были классифицированы как отрицательные, но фактически положительные).

FP (False Positives) – отрицательные примеры, классифицированные как положительные (ошибка II рода); Это ложное обнаружение, т.к. при отсутствии события ошибочно выносится решение о его присутствии (ложно положительные случаи) (то есть объекты, которые были классифицированы как положительные, но фактически отрицательные).

Accuracy – широко используемая и простаяя для понимания метрика. Это отношение всех правильных ответов к общему числу всех предсказываемых объектов (или к общему числу всех ответов). Иногда бывает неинформативной.

𝐴ccuracy = (𝑇𝑃 + 𝑇𝑁) / (TP + TN + FP + FN)

Точность (precision) - это доля прогнозируемых положительных результатов, которые являются действительно истинно положительными результатами для всех положительно предсказанных объектов. Точность показывает способность классификатора не помечать образец как положительный, когда он является отрицательным.

Точность = 𝑇𝑃 / (𝑇𝑃 + FP)

Полнота (recall) – отношение всех истинно положительно предсказанных объектов к общему количеству действительно положительных. То есть, полнота показывает сколько образцов из всех положительных примеров были классифицированы правильно. Чем выше значение полноты, тем меньше положительных примеров пропущено в классификации. То есть полнота показывает способность классификатора находить все положительные объекты.

Полнота = 𝑇𝑃 / (TP + FN)

Специфичность (specificity) – отношение между верно классифицированных негативных экземпляров к числу всех негативных экземпляров.

Специфичность = 𝑇𝑁 / (𝑇𝑁 + FP)

In [19]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report 
#импортируем описанные выше метрики

In [20]:
precision_score(y, predict) # точность модели, во всех случаях мы сравниваем известные значения целевой переменной y и 
# предсказанные значения 'predict'

0.6863636363636364

In [21]:
recall_score(y, predict) # полнота модели

0.5206896551724138

In [22]:
accuracy_score(y, predict) # по сути это доля верных ответов

0.7086834733893558

In [23]:
print(classification_report(y, predict)) 
# общий отчет с метриками

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       424
           1       0.69      0.52      0.59       290

    accuracy                           0.71       714
   macro avg       0.70      0.68      0.68       714
weighted avg       0.71      0.71      0.70       714



In [24]:
# Хотя модель и стала адекватной, однако мы совершаем еще одну методическую ошибку.
# Мы обучаем машину и проверяем ее на одних и тех же данных.
# В реальности мы обучаем на одних данных, а проверять машину должны на других данных,
# которые она еще не видела.

In [25]:
from sklearn.model_selection import train_test_split 
# импортируем функцию, которая поможет нам разбить наш датафрейм на обучающую и тестовую выборки.

![Image](http://scott.fortmann-roe.com/docs/docs/MeasuringError/holdout.png)

In [26]:
# Наличие random_state делает перемешивание выборки постоянным от запуска к запуску. Без нее перемешивание будет разным
# при каждом запуске кода
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)
# в итоге мы получаем 4 множества: обучающую и тестовую выборки без ответов и два множества ответов (y_train, y_test),
# которые поделены в том же соотношении, что и X_train и X_test.
# соотношение в котором нужно разделить датафрейм на обучающую и тестовую выборки задается test_size=0.33
# на вход подаются два множества: датафрейм X и ответы, которых нет в датафрейме y (этот тот самый столбец 'Survived')

In [27]:
X_train.head() #обучающая выборка

,PassengerId,Pclass,Age,SibSp,Parch,Fare
271,272,3,25.0,0,0,0.0000
853,854,1,16.0,0,1,39.4000
696,697,3,44.0,0,0,8.0500
518,519,2,36.0,1,0,26.0000
609,610,1,40.0,0,0,153.4625


In [28]:
X_train.shape #параметры обучающей выборки

(478, 6)

In [29]:
X_test.shape #параметры тестовой выборки

(236, 6)

In [30]:
(X_test+X_train).shape #если соединить обе выборки вместе, то получим исходное множество

(714, 6)

In [31]:
print (y_train.shape); print (y_test.shape); #для целевой функции все аналогично

(478,)
(236,)


In [33]:
# Повторяем код обучения и предсказывания, но обучаем на одном куске, а предсказываем на другом
model = LogisticRegression(max_iter=10000)       # инициализируем модель
model.fit(X_train, y_train)                      # обучаем модель
predict = model.predict(X_test)                  # делаем предсказание

In [34]:
from sklearn.metrics import accuracy_score, f1_score #рассчитаем метрики

print('precision', precision_score(y_test, predict))
print('recall', recall_score(y_test, predict))
print('accuracy', accuracy_score(y_test, predict))
print('f1', f1_score(y_test, predict))

precision 0.7424242424242424
recall 0.494949494949495
accuracy 0.7161016949152542
f1 0.5939393939393939
